In [1]:
!pip install -U boto3 sagemaker

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/50/58/2324b65441c0f83013d27ad8e4b614688d4dd79237ff43d1488dfcc01f3b/boto3-1.28.64-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.8/902.8 kB 18.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for botocore<1.32.0,>=1.31.64 from https://files.pythonhosted.org/packages/d0/68/6a9c405bc6c6e7d832743a458c87f21cee393ef2cf32437a0a3a07cd0ae9/botocore-1.31.64-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 34.3 MB/s eta 0:00:00:00:010:01
  Created wheel for sagemaker: filename=sagemaker-2.192.1-py2.py3-none-any.whl size=1206240 sha256=79e6e884f9e4b80d715e056bbb3b86a8a4594c80a319f7ecf7dcaf2d03bcdb68
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ba/74/23/fc5c71a7784765c4f3a262981272f28e4b5f812b74d20e1ebe
Succes

In [2]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
region = sagemaker.Session().boto_region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

In [5]:
identity = pd.read_csv("./datasets/identity.csv")
transaction = pd.read_csv("./datasets/transaction.csv")
pd.set_option("display.max_columns", 500)
identity.head()

,TransactionID,IpAddress,PhoneNo,DeviceID
0,f9980c5d-e8bf-4431-9ddf-15b4e7a9d7ff,104.32.122.67,7993004217,657817920030
1,011955b3-14fa-4342-85f4-3a9c3759b7d7,164.180.250.14,6712423669,8984812524268
2,a61677ac-a0cb-4113-9409-87bcfe7e6e44,99.31.120.70,+1-353-046-6749x00004,3092060466768
3,f46e41fc-5d06-4f60-b730-a5d7ecec90f1,10.225.43.49,125-534-6633x1775,2516617099683
4,4cc7b6f5-9579-4149-80d8-8cb6c807ee6c,218.46.8.47,(374)448-9917,7683358940269


In [6]:
transaction.head()

,TransactionID,TransactionDT,card_no,card_type,email_domain,ProductCD,TransactionAmt,isFraud
0,4e481ebc-c0a3-4682-92bb-852a51707c3f,2023-01-15 00:00:20,3572146632952415,JCB 16 digit,hotmail.com,45,669,1
1,6023c48d-1a0b-4639-b2b9-9dd84e5624a3,2023-01-15 00:00:44,5345027347086361,Maestro,mathis.com,Y,2924,0
2,9b81b063-6d23-485e-9c32-91303a0f96e0,2023-01-15 00:02:18,213121287312208,VISA 16 digit,yahoo.com,L,704,0
3,773b7155-d33b-4437-a961-8ff6577bf203,2023-01-15 00:03:01,4963405922548475459,Diners Club / Carte Blanche,rodriguez.com,45,6045,1
4,ac5b8170-be23-494f-aa58-7fe6ad5f2cc0,2023-01-15 00:03:24,3507208774395447,VISA 13 digit,jefferson.com,L,330,0


In [8]:
#import os
#from sagemaker.sklearn.processing import SKLearnProcessor
#from sklearn.model_selection import train_test_split

In [9]:
transaction["card_no4"] = transaction["card_no"].apply(lambda x: 'C' + str(x)[-4:])
transaction['TransactionDT'] = pd.to_datetime(transaction['TransactionDT'])
transaction['Week'] = transaction['TransactionDT'].dt.week
transaction['hour'] = transaction['TransactionDT'].dt.hour

df_all = pd.merge(transaction,identity,how="left",on="TransactionID")

df_all["ipLastPart"] = df_all["IpAddress"].apply(lambda x: 'I' + x.split('.')[-1])
df_all["device_no4"] = df_all["DeviceID"].apply(lambda x: 'D' + str(x)[-4:])
df_all["PhoneNo_no4"] = df_all["PhoneNo"].apply(lambda x: 'P' + str(x)[-4:])

df_all = df_all.sort_values(by=["DeviceID", 'TransactionDT'])
df_all['cumulative_fraud_per_device'] = df_all.groupby("DeviceID")["isFraud"].cumsum()
df_all['cumulative_fraud_before_date'] = df_all.groupby("DeviceID")['cumulative_fraud_per_device'].shift(fill_value=0)

/tmp/ipykernel_8458/3045001387.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  transaction['Week'] = transaction['TransactionDT'].dt.week


In [10]:
df_all.head()

,TransactionID,TransactionDT,card_no,card_type,email_domain,ProductCD,TransactionAmt,isFraud,card_no4,Week,hour,IpAddress,PhoneNo,DeviceID,ipLastPart,device_no4,PhoneNo_no4,cumulative_fraud_per_device,cumulative_fraud_before_date
38731,59b4f734-7421-4b27-b919-59bbd897fdb4,2023-01-31 05:42:27,2275217104321801,Maestro,baldwin.biz,AB,219,0,C1801,5,5,207.79.108.184,+1-477-260-4489x81116,45196238,I184,D6238,P1116,0,0
97208,d7bd86a4-5035-499d-a22f-2ec242fb0310,2023-02-24 20:48:43,2275217104321801,Maestro,hotmail.com,AB,538,0,C1801,8,20,207.79.108.184,+1-477-260-4489x81116,45196238,I184,D6238,P1116,0,0
10515,f2b287f4-a1a0-4688-966e-c9c62aab60ce,2023-01-19 09:36:49,3555131478859586,Mastercard,ochoa.biz,Y,320,0,C9586,3,9,158.209.25.188,+1-850-026-3814x071,91513366,I188,D3366,Px071,0,0
15133,97c1dec4-6a31-4e91-97f2-51fc7e979540,2023-01-21 07:10:47,3555131478859586,Mastercard,hotmail.com,AB,1866,0,C9586,3,7,158.209.25.188,+1-850-026-3814x071,91513366,I188,D3366,Px071,0,0
15875,f703408e-6f80-4868-bb9f-7cc524db2869,2023-01-21 14:41:24,3555131478859586,Mastercard,mitchell.com,AB,311,0,C9586,3,14,158.209.25.188,+1-850-026-3814x071,91513366,I188,D3366,Px071,0,0


In [11]:
df_all.drop(["TransactionID","TransactionDT","card_no","IpAddress","PhoneNo","DeviceID","cumulative_fraud_per_device"],axis=1,inplace=True)

In [12]:
df_all.head()

,card_type,email_domain,ProductCD,TransactionAmt,isFraud,card_no4,Week,hour,ipLastPart,device_no4,PhoneNo_no4,cumulative_fraud_before_date
38731,Maestro,baldwin.biz,AB,219,0,C1801,5,5,I184,D6238,P1116,0
97208,Maestro,hotmail.com,AB,538,0,C1801,8,20,I184,D6238,P1116,0
10515,Mastercard,ochoa.biz,Y,320,0,C9586,3,9,I188,D3366,Px071,0
15133,Mastercard,hotmail.com,AB,1866,0,C9586,3,7,I188,D3366,Px071,0
15875,Mastercard,mitchell.com,AB,311,0,C9586,3,14,I188,D3366,Px071,0


In [13]:
## That is super simplistic of course....
cat_cols = ["card_type","email_domain","ProductCD","card_no4","ipLastPart","device_no4","PhoneNo_no4"]
num_cols = [c for c in df_all if c not in cat_cols]

def compute_top_10_values(df, columns):
    df_copy = df.copy()

    for col in columns:
        # Calculate the top 10 most frequent values
        top_10_values = df_copy[col].value_counts().nlargest(20).index.tolist()

        # Replace values not in the top 20 with 'other'
        df_copy[col] = df_copy[col].apply(lambda x: x if x in top_10_values else 'other')

    return df_copy

df_all_recoded = compute_top_10_values(df_all, cat_cols)

df_prep = pd.get_dummies(df_all_recoded) #df_all[cat_cols]
df_prep = pd.concat(
    [df_prep,df_all[num_cols]], axis=1
)
df_prep = df_prep.astype(float)

In [14]:
df_prep.head()

,TransactionAmt,isFraud,Week,hour,cumulative_fraud_before_date,card_type_American Express,card_type_Diners Club / Carte Blanche,card_type_Discover,card_type_JCB 15 digit,card_type_JCB 16 digit,card_type_Maestro,card_type_Mastercard,card_type_VISA 13 digit,card_type_VISA 16 digit,card_type_VISA 19 digit,email_domain_anderson.com,email_domain_brown.com,email_domain_davis.com,email_domain_garcia.com,email_domain_gmail.com,email_domain_hernandez.com,email_domain_hotmail.com,email_domain_johnson.com,email_domain_jones.com,email_domain_lee.com,email_domain_lopez.com,email_domain_martinez.com,email_domain_miller.com,email_domain_moore.com,email_domain_other,email_domain_rodriguez.com,email_domain_smith.com,email_domain_taylor.com,email_domain_williams.com,email_domain_wilson.com,email_domain_yahoo.com,ProductCD_45,ProductCD_AB,ProductCD_L,ProductCD_T,ProductCD_Y,card_no4_C0303,card_no4_C1070,card_no4_C1824,card_no4_C1943,card_no4_C1987,card_no4_C2225,card_no4_C2547,card_no4_C3287,card_no4_C3457,card_no4_C3611,card_no4_C4548,card_no4_C4576,card_no4_C6062,card_no4_C6572,card_no4_C7907,card_no4_C8169,card_no4_C8544,card_no4_C9035,card_no4_C9308,card_no4_C9753,card_no4_other,ipLastPart_I11,ipLastPart_I110,ipLastPart_I114,ipLastPart_I126,ipLastPart_I137,ipLastPart_I139,ipLastPart_I14,ipLastPart_I149,ipLastPart_I155,ipLastPart_I165,ipLastPart_I183,ipLastPart_I212,ipLastPart_I227,ipLastPart_I245,ipLastPart_I28,ipLastPart_I33,ipLastPart_I43,ipLastPart_I63,ipLastPart_I67,ipLastPart_I7,ipLastPart_other,device_no4_D0077,device_no4_D0243,device_no4_D1004,device_no4_D1270,device_no4_D1873,device_no4_D2222,device_no4_D2337,device_no4_D4213,device_no4_D4606,device_no4_D4669,device_no4_D4754,device_no4_D5280,device_no4_D5617,device_no4_D5984,device_no4_D6616,device_no4_D6956,device_no4_D7637,device_no4_D7661,device_no4_D7711,device_no4_D8067,device_no4_other,PhoneNo_no4_P1040,PhoneNo_no4_P4757,PhoneNo_no4_Px042,PhoneNo_no4_Px053,PhoneNo_no4_Px064,PhoneNo_no4_Px085,PhoneNo_no4_Px191,PhoneNo_no4_Px207,PhoneNo_no4_Px233,PhoneNo_no4_Px237,PhoneNo_no4_Px302,PhoneNo_no4_Px342,PhoneNo_no4_Px389,PhoneNo_no4_Px501,PhoneNo_no4_Px523,PhoneNo_no4_Px527,PhoneNo_no4_Px638,PhoneNo_no4_Px879,PhoneNo_no4_Px911,PhoneNo_no4_Px919,PhoneNo_no4_other,TransactionAmt,isFraud,Week,hour,cumulative_fraud_before_date
38731,219.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,219.0,0.0,5.0,5.0,0.0
97208,538.0,0.0,8.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,538.0,0.0,8.0,20.0,0.0
10515,320.0,0.0,3.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,320.0,0.0,3.0,9.0,0.0
15133,1866.0,0.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0

In [18]:
train, test = np.split(
    df_prep.sample(frac=1, random_state=42),
    [int(0.7 * len(df_prep))],
)
train.to_csv("./datasets/train.csv", header=False, index=False)
test.to_csv("./datasets/test.csv", header=False, index=False)

In [25]:
prefix = "ngdatasandbox/fraud-detection"
boto3.Session().resource("s3").Bucket(bucket).Object(os.path.join(prefix, "train.csv")).upload_file("./datasets/train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(os.path.join(prefix, "test.csv")).upload_file("./datasets/test.csv")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [26]:
container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, "1.7-1")
display(container)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'659782779980.dkr.ecr.eu-west-3.amazonaws.com/sagemaker-xgboost:1.7-1'

In [27]:
s3_input_train = TrainingInput(
    s3_data="s3://{}/{}/train".format(bucket, prefix), content_type="csv"
)
s3_input_validation = TrainingInput(
    s3_data="s3://{}/{}/test/".format(bucket, prefix), content_type="csv"
)

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=3,
    subsample=0.8,
    verbosity=0,
    objective="binary:logistic",
    num_round=100,
)

xgb.fit({"train": s3_input_train, "validation": s3_input_validation})

In [ ]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ""
    for array in split_array:
        predictions = "".join([predictions, xgb_predictor.predict(array).decode("utf-8")])

    return predictions.split("\n")[:-1]


predictions = predict(test.to_numpy()[:, 1:])

In [ ]:
predictions = np.array([float(num) for num in predictions])
print(predictions)

In [ ]:
pd.crosstab(
    index=test.iloc[:, 0],
    columns=np.round(predictions),
    rownames=["actual"],
    colnames=["predictions"],
)

In [ ]:
plt.hist(predictions)
plt.xlabel("Predicted fraud probability")
plt.ylabel("Number of users")
plt.show()

In [ ]:
pd.crosstab(index=test.iloc[:, 0], columns=np.where(predictions > 0.3, 1, 0))

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
for c in cutoffs:
    costs.append(
        np.sum(
            np.sum(
                np.array([[0, 100], [500, 100]])
                * pd.crosstab(index=test.iloc[:, 0], columns=np.where(predictions > c, 1, 0))
            )
        )
    )

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.xlabel("Cutoff")
plt.ylabel("Cost")
plt.show()

In [ ]:
print(
    "Cost is minimized near a cutoff of:",
    cutoffs[np.argmin(costs)],
    "for a cost of:",
    np.min(costs),
)

In [ ]:
xgb_predictor.delete_endpoint()